#Texas hold 'em Poker Player AI Design
#### AM207 Advanced Scientific Computing: Stochastic Optimization Methods
#### By Yung-Jen Cheng, Peiheng Hu, Sail Wu, Xide Xia

github address: https://github.com/quezacot/AM207Final

website: https://sites.google.com/site/texasholdemdemo/

##Abstract

Poker, Texas hold 'em, is a game of imperfect information, making it impossible for 
anyone to conclude the final outcome of the hand. Admittedly, we can derive nearly 
deterministic probability of a given hand from millions of simulation results. However, 
the way how people react and the psychological components involved in betting make 
poker one of the most popular card games in the world. This project aims to implement 
a system which allows the designed computer program to model the style of the player, 
obtain observations of the style of that player and update the style of the player to play 
in the game of heads-up unlimited Texas hold 'em.

##Introduction

In this project we have designed a Heads-up unlimited AI Texas hold 'em poker player by using both statisitcally deterministic model and Bayesian analysis on opponent past betting behaviors.In the **Approach** section, we outlined our strategy to design such a complicated system. We explained our methods to create a rational poker AI in greater details in  **Deterministic Approach** section, and Hidden Makov Model and Bayesian MCMC in **Human Behavior Adjustment Approach** section. **Deterministic Approach** section covers some fundamental probability and statisitics which are used in calculating optimal betting size in the poker game, and **Human Behavior Adjustment Approach** involves hidden Markov model (HMM) and Bayesian MCMC modeling techniques which are used to make statistical inference on human player's betting behaviors.

## Approach

The following flow chart illustrate our approach to design an AI Texas hold 'em poker player:

![](approach.png)

The top half of the flow charts reflects the deterministic approach, which allows AI to use its excessive computing power to calculate the winning probability and make the statistically optimal betting decisions. Detailed processes are explained in the winning probability calculation and rational play flow design sub-sections. 

Besides designing a completely rational AI player, we analyzed the human player's past betting records and modified the determinisitc approach in the hope of increasing AI's expected returns. Specifically, we used HMM to estimate opponent's hole cards strengths and Bayesian modeling with MCMC to predict opponent's bluff tendency in each of his actions.

## Deterministic Approach

###Poker Simulator Design

To provide interative experience between user and computer with the capability for the computer to automatically keep track of human player's previous behaviors and then apply stochastic methods on his behaviors, a poker simulator that contains objects such as game and players are created.

In object Game, it stores the two players playing the game and keep track of the game progress from randomly shuffling and distributing cards to terminating the game and determineing the winner.

In object Player, it contains all the information that player can gain access to, such as what card he has in hand, how much money he has in hand, and also how much money he contributed to the current pot size. Beside these attributes, Player can be either a computer or a user. In this way, the simulator provides the functionality to have two computer players play against each other, two human play with each other, and more importantly, one user and one computer play against each other with the computer capable of learning the user's behavior and making smarter actions.

In the game process, everytime of when the user makes an action, we will store a tuple that contains the information of that action in playrs betHistory attribute. The tuple is created with three values, 1. action type, 2. calculated winning probability given the current stage cards in hand and on the table, and 3. betting percentage, which is the betting amount divided by the current total pot size. In this way, the simulator can record every action of the user and pass these past information to the stochastic models for analysis to figure out how good the cards are in user's hand and also how tendency that the user will bluff.

###Winning Probability Calculation

The winning probability is the probability to win this game, given hole cards and the board cards are revealed on the table. In the river stage, when all five bard cards are known, we compute this probability by permuting all possible opponent's hole cards and count winning times over total $\frac{45\times 44}{2}=990$ combinations. 

In the turn stage, the method is similar but we need to permute one extra card since only four board cards are known. The computation expenses for the winning probability estimation at the turn stage is less than 1 second. The following figure illustrate the process:

![](winning_prob.png)

In the flop stage, the number of permutations grows to $\frac{47\times46}{2}\times \frac{45\times44}{2} = 1070190$, which is too large to compute on the fly (computation requires more than 40 seconds to compute). Thus, we only sample one tenth of opponent’s hole cards to approximate it.

In the preflop stage, we used following tables from the poker statisitcs website, http://seoblackhat.com/texas-hold-em-poker-statistics/ to compute AI's winning probability.

![](preflop_win.png)

###Rational Play Flow Design

With the probability of winning computed from permuting all the combinations, a fixed/deterministic strategy can be deduced.

Scenario 1 winning probability > 0.5:  Better off to raise more money. The bet amount is calculated based on the winning probability.

Scenario 2 winning probability < 0.5:  Denote winning probability as P, current money in pot is potSize. We want to solve the optimal value to raise, which is x.

![""](action.PNG)

## Human Behavior Adjustment Approach

There is a famous quote about playing Poker, "A player who never bluffs will never win a big pot". Inspired by that, many players tend to bluff frequently in the Poker table. Especially in a heads-up game, players tend to involve in the game with a wider range of hole cards. In order to make our AI player act like a human, we implemented the Bayesian MCMC method to estimate the opponent's bluff tendency at each action and Hidden Markov Model method to estimate the opponent's hole hand strength. As such, AI's rational betting strategies could be adjusted according to its knowledge of opponent's past behaniors and make higher expected returns from the game.

###Hidden Markov Model Hand Strength Estimation

We used the following charts to model the strength of opponent's hole hand, based on the revealed information we got from the historical records. Since there are more than 2 million combinations of cards in the poker game, we discretized the states based on each opponent's hand winning probability, and classified them into {Weak,Medium,Strong}: "Weak" means hand winning probability is less than 33%, "Medium" 33% to 66%, and "Strong" larger than 66%. 

![](hmm_1.png)

The transition probability is simplified as in the following chart, which illustrate how likely the hidden state transits among each other in the game.

![](hmm_2.png)

The observed states in HMM model will be represented by the pairs of action and betting size relative to the pot size. Thus, Observed State, O:{(Call, L), (Call,M), (Call, H), (Raise,L), (Raise,M), (Raise,H)}. Similar to the hole hand strength, we discretized the betting size into low (L), medium (M) and high (H) 3 different classes as well. The emission probability table below is used to calculate the emssion probability for each hidden state and observed state. Through out the game, we used historical records to update the table and compute the probability sequentially.

![](hmm_3.png)

We implemented Vitorbi Algorithm to estimate the player’s most likely hand strength in the current stage. After estimating the player’s hidden hand strength, we adjusted the winning probability accordingly. Please refer to **Strength_HMM.py** for the code.

###Bluff Tendency  Bayesian MCMC Estimation

We model a player’s bluff status with probability with probability $f(\theta_{1}+\theta_{2}t)$
conditionally on $\Theta = (\theta_1, \theta_2)$. We used $f(z) = 1/(1 + \exp(-z))$ to denote a player’s likelihood to bluff in each action. $t$ is the player’s betting size relative to pot size. Due to limited prior information for each player, we chose a very flat prior: $\theta_1,\theta_2\sim\mathcal{N}(0,10^2)$.

Thus, Prior can be expressed as: 

$$P(\theta_1,\theta_2)=P(\theta_1)\times P(\theta_2)
=\frac{1}{\sqrt{2\pi}\sigma_1\sqrt{2\pi}\sigma_2}\exp{(-\frac{\theta_1^2 }{2\sigma_1^2}-\frac{\theta_2^2 }{2\sigma_2^2})}
=\frac{1}{2\pi\sigma_1\sigma_2}\exp{(-\frac{\theta_1^2 }{2\sigma_1^2}-\frac{\theta_2^2 }{2\sigma_2^2})} $$

Where $\sigma_1=\sigma_2=10$


We Expressed the likelihood for each action as:

$$Likelihood_i= \frac{1}{1+\exp(-\theta_1-\theta_2t_i)}^{y_i}\times\frac{\exp(-\theta_1-\theta_2t_i)}{1+\exp(-\theta_1-\theta_2t_i)}^{1-y_i}$$

Where $y_i \in \{0,1\}$ indicates if the player is bluffing at each action based on if his current winning probability greater than $0.5$. The posterior likelihood can be expressed as :

$$P(\Theta|D) \propto P(D|\Theta) \times P(\Theta) $$

$$P(\Theta|D)\propto \prod_{i=1}^N (\frac{1}{1+\exp(-\theta_1-\theta_2t_i)}^{y_i}\times\frac{\exp(-\theta_1-\theta_2t_i)}{1+\exp(-\theta_1-\theta_2t_i)}^{1-y_i})\times \exp{(-\frac{\theta_1^2 }{2\sigma_1^2}-\frac{\theta_2^2 }{2\sigma_2^2})} $$

We then used MCMC rejection sampling to estimate the player’s bluff probability, and incorporated that to adjust our winning probability in the overall betting strategies. Please refer to **Bluff_Bayesian_new.py** for the code.

###Integrating Human Beharvior to Deterministic Strategies

The following chart shows the adjument AI player made to calculate the winning probability after using HMM adjustment:
![](integration.png)

Deterministic calculation shows the winning probability is $\pi_0$, which equals to the shaded area to the left of $\pi_0$. The adjustment of winning probability calculated based on one of the three states below. 

We also believe that if the opponent has a higher probability to bluff, we will be less certain about the results from HMM estimation and rely more on the uniform distribution of his hole hand. Thus, the adjusted winning probability will be $\pi_{adj}=w\pi_0+(1-w)\pi_{state}$ 
Where $w$ is the opponent's probability to make bluff at his turn.

## Conclusion

Our project successfully created an AI poker player that can determine the optimal betting strategies that based on both statistics and human behaviors to maximize its profits. We obtained the statistically optimal betting decision based on calculating the winning percentage for each given hand after permutating all the possible scenarios of the opponent hands. The human behaviors are captured by the historical records of the players and we used HMM to estimate player’s hand strength and Bayesian MCMC to deduce whether the player is bluffing at each action. We have conducted thousands of runs to be tested on both AI poker players for debugging and parameter tunings. Furthermore, unlike other conventional game simulators, our code does not require intensive training time, but produces computationally robust results. 

## References

1. Preflop hand strength: http://seoblackhat.com/texas-hold-em-poker-statistics/ 
2. Poker betting strategies: http://www.pokerlistings.com/strategy/sizing-your-bets-properly

## Appendix: Software Install Instructions

Download all code and run gameflow.py to play with our AI.

**$ python gameflow.py**

Our code files are splitted to following 10 files.

**deal_compare.py** This file has functions to determine the type of five cards, compare two hands of five cards, and find out the largest five cards from seven cards (board cards + hole cards). The following functions are used to determine the type of five cards: flush(fivecards); straight(fivecards); straightflush(fivecards); fourkind(fivecards); fullhouse(fivecards); threekind(fivecards); twopairs(fivecards); onepair(fivecards) The input is a list of five cards. The output will be a Boolean indicating whether these five cards belong to this kind, and the sorted cards to compare the same type of cards. e.g. ['KS','TH', '3C', 'KD', '3S'] is five cards of two pairs. The output sorted list is [K, 3, T], which means the larger pair, the smaller pair and the rest high card. This is used to compare two hands of five cards with the same type. compare(fivecards1,fivecards2) takes input of two hands of five cards and output the larger one. maxeach(boardcards,holecards) takes inputs of five board cards and two hole cards, and output the best five cards.

**createCompareDict.ipynb** This file generates a table of all possible five cards and sorts them in the order of cards' strength. The output is a pickle file allCombDict_new.p. The strength is defined as a value that the larger five cards have larger strength. If two five cards are tie then they will have the same strength.

**each_compare.py** This file has the same functions in deal_compare.py but eliminates the functions that do not need after the table allCombDict_new.p is generated.

**initial_52.py** This file has convenient utilities that transfer numbers to cards, and generate a deck of all 52 cards.

**winningprob.py** This file computes the deterministic winning probabilities in flop, turn, and river stages. It has the following functions: bestfive() takes input of seven cards and output the best five cards and its strength. This function uses the table allCombDict_new.p we generated. winp_river() takes inputs of five board cards and two hole cards. Output is the probability to win. When samplerate is set, this function only runs through part of the combinations of opponent's hole cards. winp_turn(), winp_flop()takes inputs of four, three board cards respectively and two hole cards. Output is the probability to win.

**Bluff_Bayesian_new.py** This file uses Bayesian to compute the bluff probability of the opponent. Import this file and use fuction Bluff(Hist_Record) to compute this probability. The input argument is the betting history of the opponent. The output is a probability between [0,1]

**Strength_HMM.py** This file uses HMM to estimate the strength of opponent's hole cards. Import this file and use HMM_state(Hist_Record) to estimate the strength state of opponent's hole cards. The input argument is the betting history of the opponent. The outpur is a state of opponent's hole cards. The possible states are 'Low', 'Medium', 'High', or 'No_State'. 'No_State' is returned when there is no betting history to run HMM.

**roundcontrol.py** This file has two classes 'player' and 'roundcontrol.' player class is to store all parameters and handles all actions related to a player, such as their hole cards, money in hand, bet history and betting action. roundcontrol class is to store all parameters and handles all actions related to the table, such the money in pot, the board cards and to advance game stage.

**preflop_table.py** This stores the winning probabilities when only two hole cards are known. card_prob() takes input of two hole cards the an index to indicate the case whether the player is dealer.

**preflop.py** This file controls the game of the preflop stage. (only two hole cards are known) preflop() takes the input of roundcontrol that contains all information and an indicator of which player is the dealer. preflopMakeAction() is to handle the bet action. The human player will be asked to input when it is his turn to bet. The computer's will use its AI to determine the bet.

**postFlop.py** This file controls the game of the flop, turn, and river stage. (three, four, five board cards are known respectively) afterPreFlop() deals with the result of preflop(), stores the necessary changes and advances to flop stage. postFlop() takes the input of roundcontrol that contains all information and an indicator of which player is the dealer. postflopMakeAction() is to handle the bet action. The human player will be asked to input when it is his turn to bet. The computer's will use its AI to determinge the bet. adjustpibystate() uses the estimation that computes from Bluff_Bayesian_new.py and Strength_HMM.py

**gameflow.py** This file combines preflop.py and postFlop.py to operatie the over all game flow. It is also responsible to distribute the money to winner in the end of each game and restart the next game if the human player wants to.